In [ ]:

# necessary imports
import numpy as np
import csv
import os
from google.colab import drive 
import matplotlib.pyplot as plt
import random # Added by Caleb
import math # Added by Caleb
import os
import torch
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
from google.colab import drive 
drive.mount('/content/drive/') ## mount to drive. This will ask for permission to access your Google drive each time

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data_folder_path = "/content/drive/Shareddrives/Neural Nets/Competition/ECE542_sp2022_Project_TerrainRecognition/" # path into Lobton's directory 
type_of_data = "TestData" # Read in the type of data you want. Options are either:  'TrainingData' or 'TestData'
list_of_files = os.listdir(data_folder_path + type_of_data) # List everything in the directory at place 2022_Project_TerrainRecognition/TrainingData or /TestData (from line above)
list_of_files.sort() # Sort the list of files
print(list_of_files) # Print out the list of files
# need to load all this data in for augmentation (only the x data, but need to match what y data it connects with)
# we need to figure out what data we want to use too. 

['subject_009_01__x.csv', 'subject_009_01__x_time.csv', 'subject_009_01__y_time.csv', 'subject_010_01__x.csv', 'subject_010_01__x_time.csv', 'subject_010_01__y_time.csv', 'subject_011_01__x.csv', 'subject_011_01__x_time.csv', 'subject_011_01__y_time.csv', 'subject_012_01__x.csv', 'subject_012_01__x_time.csv', 'subject_012_01__y_time.csv']


In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, x_data, y_data, img_dir=None, transform=None, target_transform=None):
        self.img_labels = y_data
        self.img_dir = img_dir
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.x_data[idx]
        label = self.y_data[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [ ]:
# Code block created by Caleb
# Goal: Session class that holds subject and session number, and also returns the data for that session

# This class can return the name of the file that holds the requested data
    # Example of how to use:
    # newSession = Session(4, 5)
    # newStr = newSession.xTimeName()
    # print(newStr)
    # # Prints out "subject_004_05__x.csv"

class Session:
  def __init__(self, subject_number, session_number):
    self.subject_number = subject_number  # Initialize subject number
    self.session_number = session_number  # Initialize session number

  # Each of the following member functions return the name of the specified file for that subject and session number of the Session:

  def xTimeName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__x_time.csv"

  def yTimeName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__y_time.csv"

  def xDataName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__x.csv"

  def yDataName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__y.csv"


  # This function input is the Session object that contians the subject and session numbers
  def getXDataFromFile(self):
    x_data_path = data_folder_path + "TestData/" + self.xDataName() # Get the path to the x_data file 
    x_data = np.genfromtxt(x_data_path, delimiter=',')  # Read the data in from the text file
    return x_data  # Return the data array
    # This function returns the array of all the six x values

  def getXTimeFromFile(self):
    x_time_path = data_folder_path + "TestData/" + self.xTimeName() # Get the path to the x_data file 
    x_time = np.genfromtxt(x_time_path, delimiter=',')  # Read the data in from the text file
    return x_time  # Return the data array
    # This function returns the array of just the x time values

  def getYTimeFromFile(self):
    y_time_path = data_folder_path + "TestData/" + self.yTimeName() # Get the path to the x_data file 
    y_time = np.genfromtxt(y_time_path, delimiter=',')  # Read the data in from the text file
    return y_time  # Return the data array
    # This function returns the array of just the y time values




In [ ]:
# Code block created by Caleb:
# Goal is to create list of each session, with a unique session id (session id made up of subject number and session number)

list_of_sessions = []

for file_name in list_of_files:
  # Extract subject and session number from file_name:
  new_subject_number = int(file_name[8:11]) # Since all provided subject and session numbers are single digit, we can 
  new_session_number = int(file_name[12:14]) #   pick them out of the file name by grabbing an exact character number.
  


  # Check if I should add this session to the list of sessions
  should_add_session = True # Create a variable that by default should add the session
  for session in list_of_sessions: # Loop through the sessions in the list of sessions (list I may need to add it to)
    comp_subject_number = session.subject_number # Get both new session and subject number
    comp_session_number = session.session_number #   save them as comp_
    if (comp_session_number == new_session_number) and (comp_subject_number == new_subject_number): # If comp session and subject are equal to new subject and session
      should_add_session = False # In this case, that would mean that I should /not/ add the new session

  if should_add_session: # If I should add the session
    newSession = Session(new_subject_number, new_session_number) # Initialize the new session object with the new subject and session values
    list_of_sessions.append(newSession) # Append the new session to the list of sessions

# Now list_of_sessions has a list of Session objects for each of the sessions

# Print out the list of the sessions in list_of_sessions:
# for session in list_of_sessions:
#   newStr = "Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number)
#   print(newStr) 


In [ ]:
# Code block created by Caleb
# Goal: Create a function that will take in an array of size [(length of file) x 6] and pad it with zeros for image size

# This function pads with image width - 1 rows of zeros
def padData(data, image_width):
  dim = [image_width - 1,6] # The dimensions of the zeros will be the width of the image and six wide
  zeroArray = np.zeros(dim) # Create an array with zeros at the beginning of the correct size
  return np.concatenate((zeroArray, data)) # Concatonate the two arrays together with the zeros at the beginning

# Print out the data for a small data array to see where the zeros go
data = np.array([[1, 2, 3, 4, 5, 6], [3, 4, 5, 6, 7, 5]])
paddedData = padData(data, 3) 
print(data)


[[1 2 3 4 5 6]
 [3 4 5 6 7 5]]


In [ ]:
# Code block created by Caleb
# Goal: Step three of the whiteboard photo, so create "images" from padded data

# This function takes in the padded data (an array  [(image_width + 40*session lenght).x 6] ) and an integer of the width of an image
def createImages(paddedData, image_width):
  all_images = [] # Create an empty array that will eventually hold all the images for this session's data
  number_of_images = paddedData.shape[0] - image_width + 1  # The number of images I should create is padded_data - (image_width - 1)
  for i in range(number_of_images): # Loop through i for each of the images I need to create
    all_images.append(paddedData[i:(i+image_width)])  # Grab out the array from the padded data equal to the image_width and starting at i, and append it to all images
  return np.asarray(all_images) 


# This example uses the 'data' array from the code block above to pass into this function to see if it successfully creates two images
#   One should have two rows of zeros (because it gets all the padding) and the next should have one row of zeros
images = createImages(paddedData, 3)
print(images)

[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]]

 [[0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]
  [3. 4. 5. 6. 7. 5.]]]


In [ ]:
# I will use the first session for this example:
example_session = list_of_sessions[0]

example_x_data = example_session.getXDataFromFile()
example_x_time = example_session.getXTimeFromFile()
#example_y_data = example_session.getYDataFromFile() # y data does not exist here
example_y_time = example_session.getYTimeFromFile()

# No need to extrapolate the ys 
#xample_extrapolated_y = extrapolatedYs(example_x_time, example_y_time, example_y_data)

# Now the x data and the extrapolate_y have the correct sizes, so y is the predictions we want
print("New y size, and then x data shape")

print(example_x_data.shape)
print("- - -")

# Now I will show the examples of transforming the x values into images. 
example_image_width = 60 # You can play around with the width of the image
example_x_padded = padData(example_x_data, example_image_width)
print("With padding:")
print(example_x_padded.shape)
print("- - -")

example_x_images = createImages(example_x_padded, example_image_width)
print("Shape of images, so should be back down to number of x values, with image_size x 6 secondary dimensions")
print(example_x_images.shape)
print("- - -")
print("Printing first images")
# print(example_x_images[0])
# print(example_x_images[1])

# So the data we're treating just like images for classification are:
# example_x_images          as the images
# example_extrapolated_y    as the labels

New y size, and then x data shape
(37991, 6)
- - -
With padding:
(38050, 6)
- - -
Shape of images, so should be back down to number of x values, with image_size x 6 secondary dimensions
(37991, 60, 6)
- - -
Printing first images


In [ ]:


# Code block created by Caleb

def getImagesFromSession(session, image_width):
  image_width_number = int(image_width * 40) # This is the conversion rate between image_width in seconds and image_width the number of samples in an image

  x_data = session.getXDataFromFile() # Get all the data for that session
  x_time = session.getXTimeFromFile()
  y_time = session.getYTimeFromFile()

  
  x_padded = padData(x_data, image_width_number) # Pad the x values
  x_images = createImages(x_padded, image_width_number) # Then create images



  print("Done processing session!")


  return np.array(x_time), np.asarray(x_images), np.asarray(y_time)

#Yt_train = Yt_train.type(torch.LongTensor)
# This next line calls the function to get the data for the model with the parameters passed in.
# Remember that image width and set length are in seconds



In [ ]:
our_transform = transforms.Compose([
    # Converting RGB [0,255] to Tensor [0,1]
    transforms.ToTensor(),
    ])

def getYLabels(session, image_width, model):
  x_time, x_images, y_time = getImagesFromSession(session, image_width)
  model.eval()
  test_data = CustomImageDataset(x_images, x_time, transform=our_transform) # x_time is just filler here 
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(x_images))
  _iter = iter(test_loader)
  print(type(_iter))
  data, index = next(_iter)
  print(data.shape)
  print(type(data))
  predictions = model(data)
  return x_time, x_images, y_time, predictions



  

In [ ]:
def loadModelStateFromFile(model, filename):
  ''' Load model's state onto model passed in from a model file'''
  competition_path = "/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Models/" #path to data folder in drive
  model_file_path = os.path.join(competition_path, filename+".pt")  #joing path and adding .npy to filename passed in
  print(model_file_path)
  model.load_state_dict(torch.load(model_file_path)) # Should not need a return since the model will be modified through its object 


In [ ]:
# Custom CNN
# Defining the CNN architecture
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5, padding=2, padding_mode='zeros', dtype=float)
   ## nn.MaxPool2d()
    self.conv2 = nn.Conv2d(6, 16, 5, dtype=float)
    self.fc1 = nn.Linear(16 * 56 * 2, 800, dtype=float)
    self.fc2 = nn.Linear(800, 200, dtype=float) 
    self.fc3 = nn.Linear(200, 50, dtype=float)
    self.fc4 = nn.Linear(50, 4, dtype=float)
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = x.view(-1, 16 * 56 * 2) # At this point the feature map is 5 x 5 x 16
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [ ]:
test_session = list_of_sessions[0]
test_model = Net()

loadModelStateFromFile(test_model, "Test_Model")
test_x_time, test_x_data, test_y_time, test_predictions = getYLabels(test_session, 1.5, test_model)


/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Models/Test_Model.pt
Done processing session!
<class 'torch.utils.data.dataloader._SingleProcessDataLoaderIter'>
torch.Size([37991, 1, 60, 6])
<class 'torch.Tensor'>


In [ ]:
# "Downsample" Y labels to correpond to what needs to be submitted
def deExtrapolateYs(x_time, y_time, y_lined_up_with_x):
  correctY = [] #Empty list for the newly transformed y values
  x_time_array = np.asarray(x_time)
  for y_t in y_time: # Loop over every y_time value
    index = (np.abs(x_time_array - y_t)).argmin() # Get index of closeset x_time to the y_t you are currently looking for
    correctY.append(y_lined_up_with_x[index]) # Add the y value at that index to the transformed y's
  return np.asarray(correctY)
